## Pre-Process and download

In [ ]:
!pip install -q simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/colab/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c detection-of-ai-generated-texts

  0% 0.00/607k [00:00<?, ?B/s]
100% 607k/607k [00:00<00:00, 146MB/s]


In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import accuracy_score
import torch
import json

In [7]:
!unzip /content/detection-of-ai-generated-texts.zip

Archive:  /content/detection-of-ai-generated-texts.zip
  inflating: Sample_Submission.csv   
  inflating: test.jsonl              
  inflating: train.jsonl             


In [35]:
with open('/content/train.jsonl') as f:
    data = [json.loads(line) for line in f]

ValueError: ignored

In [37]:
with open('/content/test.jsonl') as t:
    test_data=[json.loads(line) for line in t]

In [41]:
train_df=pd.DataFrame(data)
test_df=pd.DataFrame(test_data)

In [42]:
train_df

,id,abstract,summary,generated
0,train_0001,This paper addresses the problem of evaluating...,We show that rare but catastrophic failures ma...,This paper addresses the problem of evaluating...
1,train_0002,We explore efficient neural architecture searc...,In this paper we propose a hierarchical archit...,We explore efficient neural architecture searc...
2,train_0003,We propose Episodic Backward Update - a new al...,"We propose Episodic Backward Update, a novel d...",We propose Episodic Backward Update - a new al...
3,train_0004,Animals develop novel skills not only through ...,A new RL algorithm called Interior Policy Diff...,Animals develop novel skills not only through ...
4,train_0005,Deep generative models such as Generative Adve...,We investigate the modularity of deep generati...,Deep generative models such as Generative Adve...
...,...,...,...,...
2344,train_2345,"In this paper, we present a layer-wise learnin...",Learning a better neural networks' representat...,"In this paper, we present a layer-wise learnin..."
2345,train_2346,Brain-Computer Interfaces (BCI) may help patie...,We present an open-loop brain-machine interfac...,Brain-Computer Interfaces (BCI) may help patie...
2346,train_2347,We outline the problem of concept drifts for t...,We introduce an augmented robust feature space...,We outline the problem of concept drifts for t...
2347,train_2348,Cold-start and efficiency issues of the Top-k ...,It can generate effective hash codes for effic...,Cold-start and efficiency issues of the Top-k ...


In [44]:
test_df

,id,abstract,summary1,summary2
0,test_0001,This paper addresses the problem of evaluating...,The field of few-shot learning has recently se...,We show that rare but catastrophic failures ma...
1,test_0002,We explore efficient neural architecture searc...,Convolutional neural networks (CNNs) have been...,In this paper we propose a hierarchical archit...
2,test_0003,We propose Episodic Backward Update - a new al...,"We propose Episodic Backward Update, a novel d...",One of the distinguishing aspects of human lan...
3,test_0004,Animals develop novel skills not only through ...,A new RL algorithm called Interior Policy Diff...,Distributed computing can significantly reduce...
4,test_0005,Deep generative models such as Generative Adve...,We investigate the modularity of deep generati...,We investigate methods for semi-supervised lea...
...,...,...,...,...
256,test_0257,There have been multiple attempts with variati...,Words in natural language follow a Zipfian dis...,We present a generative model that proves stat...
257,test_0258,Effective performance of neural networks depen...,We introduce amortized proximal optimization (...,"In reinforcement learning, we can learn a mode..."
258,test_0259,We propose procedures for evaluating and stren...,The use of deep learning models as priors for ...,We propose procedures for evaluating and stren...
259,test_0260,We propose a new model for making generalizabl...,"In this paper, we propose a new control framew...",We propose a new model for making generalizabl...


In [ ]:
# # Remove all HTML tag
# #a = data['b'].str.replace(r'<[^<>]*>', '', regex=True)
# train['clean_review'] = train['question'].str.replace(r'<[^<>]*>', '', regex=True)
# test['clean_review'] = test['question'].str.replace(r'<[^<>]*>', '', regex=True)

# # Remove all excessive space, special characters and new line symbols
# train['clean2_review'] = train['clean_review'].str.strip().str.lower().str.replace('\r+', ' ').str.replace('\n+',' ').str.replace('\t+',' ')
# test['clean2_review'] = test['clean_review'].str.strip().str.lower().str.replace('\r+', ' ').str.replace('\n+',' ').str.replace('\t+',' ')



In [45]:
train_df.head()

,id,abstract,summary,generated
0,train_0001,This paper addresses the problem of evaluating...,We show that rare but catastrophic failures ma...,This paper addresses the problem of evaluating...
1,train_0002,We explore efficient neural architecture searc...,In this paper we propose a hierarchical archit...,We explore efficient neural architecture searc...
2,train_0003,We propose Episodic Backward Update - a new al...,"We propose Episodic Backward Update, a novel d...",We propose Episodic Backward Update - a new al...
3,train_0004,Animals develop novel skills not only through ...,A new RL algorithm called Interior Policy Diff...,Animals develop novel skills not only through ...
4,train_0005,Deep generative models such as Generative Adve...,We investigate the modularity of deep generati...,Deep generative models such as Generative Adve...


In [47]:
train_df=train_df.drop("id",axis=1)

In [54]:
train_df

,abstract,summary,generated
0,This paper addresses the problem of evaluating...,We show that rare but catastrophic failures ma...,This paper addresses the problem of evaluating...
1,We explore efficient neural architecture searc...,In this paper we propose a hierarchical archit...,We explore efficient neural architecture searc...
2,We propose Episodic Backward Update - a new al...,"We propose Episodic Backward Update, a novel d...",We propose Episodic Backward Update - a new al...
3,Animals develop novel skills not only through ...,A new RL algorithm called Interior Policy Diff...,Animals develop novel skills not only through ...
4,Deep generative models such as Generative Adve...,We investigate the modularity of deep generati...,Deep generative models such as Generative Adve...
...,...,...,...
2344,"In this paper, we present a layer-wise learnin...",Learning a better neural networks' representat...,"In this paper, we present a layer-wise learnin..."
2345,Brain-Computer Interfaces (BCI) may help patie...,We present an open-loop brain-machine interfac...,Brain-Computer Interfaces (BCI) may help patie...
2346,We outline the problem of concept drifts for t...,We introduce an augmented robust feature space...,We outline the problem of concept drifts for t...
2347,Cold-start and efficiency issues of the Top-k ...,It can generate effective hash codes for effic...,Cold-start and efficiency issues of the Top-k ...


In [49]:
test_df

,id,abstract,summary1,summary2
0,test_0001,This paper addresses the problem of evaluating...,The field of few-shot learning has recently se...,We show that rare but catastrophic failures ma...
1,test_0002,We explore efficient neural architecture searc...,Convolutional neural networks (CNNs) have been...,In this paper we propose a hierarchical archit...
2,test_0003,We propose Episodic Backward Update - a new al...,"We propose Episodic Backward Update, a novel d...",One of the distinguishing aspects of human lan...
3,test_0004,Animals develop novel skills not only through ...,A new RL algorithm called Interior Policy Diff...,Distributed computing can significantly reduce...
4,test_0005,Deep generative models such as Generative Adve...,We investigate the modularity of deep generati...,We investigate methods for semi-supervised lea...
...,...,...,...,...
256,test_0257,There have been multiple attempts with variati...,Words in natural language follow a Zipfian dis...,We present a generative model that proves stat...
257,test_0258,Effective performance of neural networks depen...,We introduce amortized proximal optimization (...,"In reinforcement learning, we can learn a mode..."
258,test_0259,We propose procedures for evaluating and stren...,The use of deep learning models as priors for ...,We propose procedures for evaluating and stren...
259,test_0260,We propose a new model for making generalizabl...,"In this paper, we propose a new control framew...",We propose a new model for making generalizabl...


 "distilbert.encoder.layer.10.output.dense.weight"

In [ ]:
import simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = {
    "reprocess_input_data": True,
    'train_batch_size':8,
    "use_early_stopping": True,
    "early_stopping_delta": 0.01,
    "early_stopping_metric": "mcc",
    "early_stopping_metric_minimize": False,
    "early_stopping_patience": 2,
    "evaluate_during_training_steps": 500,
    "fp16":True,
    "overwrite_output_dir":True,
    'use_cached_eval_features' : False, 
    'max_seq_length':128,
    'no_cache': True,
    "num_train_epochs":5,
    
}

model = ClassificationModel(
     "distilbert", 
     "Giyaseddin/distilbert-base-cased-finetuned-fake-and-real-news-dataset",
     use_cuda=torch.cuda.is_available(),
     num_labels=2,
     args=model_args
     ,
)

model.train_model(train,show_running_loss = True, eval_df=test,acc=sklearn.metrics.accuracy_score)

  0%|          | 0/33373 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
train

,id,text,language,label
0,GZsDkNhz42u4SAzbAnUouC,"Last month, Mr. McLaren delivered a preliminar...",english,Real
1,jaRxYZgqXRNrxZiPiLT6xy,JOHANNESBURG (Reuters) - Leaders of Zimbabwe s...,english,Real
2,nYUyyPh5R5zxs8dPa4p6XY,“But our real goal is to reach the stage when ...,english,Real
3,myF2ZaAj2ekwKxuoN8fprG,BEN CARSON fearlessly explain the TRUTH behind...,english,Fake
4,DUQVYqAz4W6EZBy2CFevZq,They just spent big bucks on a new headquarter...,english,Fake
...,...,...,...,...
42445,T2TAxsd3ZX76QPCiCYk45S,Shawn Helton ศตวรรษที่ 21 Wireanother Active-S...,thai,Fake
42446,AEyZjphKDHLfrxbuzJDUMU,"“If we just impose a tariff, they’ll put recip...",english,Real
42447,A7XDwhEUCDctCcTmWXiq7w,"At a blistering speech in Columbus, Ohio (a ve...",english,Fake
42448,Q579MHsdizW54cN5fNBy22,JOHANNESBURG (Reuters) - South Africa s ruling...,english,Real


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)

  0%|          | 0/8344 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
result

{'mcc': 0.760682760399729,
 'tp': 4095,
 'tn': 3260,
 'fp': 458,
 'fn': 531,
 'auroc': 0.9516534464903217,
 'auprc': 0.9638850057491423,
 'eval_loss': 0.3183337627184128}

In [ ]:
model_outputs

array([[-2.76953125,  2.81835938],
       [ 1.99414062, -2.43554688],
       [ 2.00390625, -2.4609375 ],
       ...,
       [-2.77539062,  2.82617188],
       [ 1.90820312, -2.25195312],
       [-0.52734375,  0.765625  ]])

In [ ]:
len(wrong_predictions)

224

In [ ]:
predictions, raw_outputs = model.predict(list(test_to_predict[]))

  0%|          | 0/20540 [00:00<?, ?it/s]

  0%|          | 0/2568 [00:00<?, ?it/s]

In [ ]:
predictions

array([0, 0, 1, ..., 1, 0, 1])

In [ ]:
submit = pd.DataFrame()
submit['id'] = predict['id']
submit['label'] = predict['label']
submit.replace(0,'Fake',inplace=True)
submit["label"]=submit["label"].replace(1,"Real")
submit["label"]=submit["label"].replace(0,"Fake")



In [ ]:
submit

,id,label
0,63e2vyej6oXukEfDdoGuA5,Fake
1,ZjYG72fdvd6JmazkS32ZFx,Fake
2,ZYUhHXzaDxtncvd6nw8yiy,Real
3,HejAe5F5BGKD4HUYMT9DTg,Fake
4,5ZjAEKgz9RCsBykjtYyp4S,Fake
...,...,...
20769,5jgzkVN5YMZiBTYzYM5eLW,NaN
20770,4GrToqE8VdVCnpZzDu9AKG,NaN
20778,5hSp9PrkwVmd8fyFHDjtTG,NaN
20867,TjhMJ8gFXzrsCpcVNbvyrP,NaN


In [ ]:
submit.to_csv('submit_distil_bert.csv',header=True,index=False)

In [ ]:
predicted=pd.read_csv("/content/submit2_4_roberta.csv")

In [ ]:
predicted

,id,label
0,63e2vyej6oXukEfDdoGuA5,Real
1,ZjYG72fdvd6JmazkS32ZFx,Fake
2,ZYUhHXzaDxtncvd6nw8yiy,Fake
3,HejAe5F5BGKD4HUYMT9DTg,Fake
4,5ZjAEKgz9RCsBykjtYyp4S,Fake
...,...,...
20904,5jgzkVN5YMZiBTYzYM5eLW,Real
20905,4GrToqE8VdVCnpZzDu9AKG,Real
20906,5hSp9PrkwVmd8fyFHDjtTG,Real
20907,TjhMJ8gFXzrsCpcVNbvyrP,Real


In [ ]:
test_thai=pd.read_csv("/content/test_predicted.csv")